# XML example and exercise
****
+ study examples of accessing nodes in XML tree structure  
+ work on exercise to be completed and submitted
****
+ reference: https://docs.python.org/2.7/library/xml.etree.elementtree.html
+ data source: http://www.dbis.informatik.uni-goettingen.de/Mondial
****

In [24]:
from xml.etree import ElementTree as ET

## XML example

+ for details about tree traversal and iterators, see https://docs.python.org/2.7/library/xml.etree.elementtree.html

In [25]:
document_tree = ET.parse( './data/mondial_database_less.xml' )

In [30]:
# print names of all countries
for child in document_tree.getroot():
    print (child.find('name').text)

Albania
Greece
Macedonia
Serbia
Montenegro
Kosovo
Andorra


In [29]:
# print names of all countries and their cities
for element in document_tree.iterfind('country'):
    print ('* ' + element.find('name').text + ':'),
    capitals_string = ''
    for subelement in element.getiterator('city'):
        capitals_string += subelement.find('name').text + ', '
    print (capitals_string[:-2])

* Albania:
Tirana, Shkodër, Durrës, Vlorë, Elbasan, Korçë
* Greece:
Komotini, Kavala, Athina, Peiraias, Peristeri, Acharnes, Patra, Kozani, Kerkyra, Ioannina, Thessaloniki, Iraklio, Chania, Ermoupoli, Rhodes, Tripoli, Lamia, Chalkida, Larissa, Volos, Mytilini, Karyes
* Macedonia:
Skopje, Kumanovo
* Serbia:
Beograd, Novi Sad, Niš
* Montenegro:
Podgorica
* Kosovo:
Prishtine
* Andorra:
Andorra la Vella


****
## XML exercise

Using data in 'data/mondial_database.xml', the examples above, and refering to https://docs.python.org/2.7/library/xml.etree.elementtree.html, find

1. 10 countries with the lowest infant mortality rates
2. 10 cities with the largest population
3. 10 ethnic groups with the largest overall populations (sum of best/latest estimates over all countries)
4. name and country of a) longest river, b) largest lake and c) airport at highest elevation

In [32]:
document = ET.parse( './data/mondial_database.xml' )

In [46]:
from xml.etree import ElementTree as ET
import pandas as pd
import numpy as np
from pandas import DataFrame

In [37]:
# 10 cities with the largest population

list = []
for city in document.findall('.//city'):
    name = city.find('name')
    if  name is None:
        name = np.nan
    else:
        name = name.text
    population = city.find('population[@year="2011"]') 
    if population is None:
        population = np.nan
    else:
        population = int(population.text)
    list.append([city.attrib['id'],name, population])
df = pd.DataFrame(list, columns=['id', 'name', 'population_2011'])
df.sort_values('population_2011', ascending=False).head(10)

,id,name,population_2011
1527,cty-India-2,Mumbai,12442373.0
1582,cty-India-New-Delhi,Delhi,11034555.0
1515,cty-India-Bangalore,Bangalore,8443675.0
1000,cty-United-Kingdom-2,London,8250205.0
1382,cty-Iran-Tehran,Tehran,8154051.0
1470,cty-BD-3,Dhaka,7423137.0
1591,cty-India-7,Hyderabad,6731790.0
1505,cty-India-8,Ahmadabad,5577940.0
3056,cty-Angola-Luanda,Luanda,5000000.0
1556,cty-India-Madras,Chennai,4646732.0


In [ ]:
# 10 ethnic groups with the largest overall populations (sum of best/latest estimates over all countries)

In [38]:
ethnicgroups = dict()
for country in document.iterfind('country'):
    populations = []
    for population in country.findall('population'):
        populations.append((int(population.get('year')), int(population.text)))
    max_population = max(populations)[1]
    for ethnicgroup in country.findall('ethnicgroup'):
        population_ethnic_group = (float(ethnicgroup.get('percentage')) * max_population / 100)
        ethnicgroups[ethnicgroup.text] = ethnicgroups.setdefault(ethnicgroup.text, 0) + population_ethnic_group
df = pd.DataFrame.from_dict(ethnicgroups, orient='index')
pd.options.display.float_format = '{:20,.2f}'.format
sorted_df = df.sort_values([0],ascending=False).head(10)
print(sorted_df)

                               0
Han Chinese     1,245,058,800.00
Indo-Aryan        871,815,583.44
European          494,872,219.72
African           318,325,120.37
Dravidian         302,713,744.25
Mestizo           157,734,354.94
Bengali           146,776,916.72
Russian           131,856,996.08
Japanese          126,534,212.00
Malay             121,993,550.37


In [ ]:
# name and country of a) longest river, b) largest lake and c) airport at highest elevation

In [40]:
code_to_country_dict = {}
for country in document.iterfind('country'):
    code_to_country_dict[country.get('car_code')] = country.find('name').text

rivers = []
for river in document.iterfind('river'):
    for country in river.get('country').split():
        length = river.find('length')
        if length is None:
            length = np.nan
        else:
            length = float(length.text)
        rivers.append([river.find('name').text, length, code_to_country_dict[country]])
river_df = pd.DataFrame(rivers, columns=['name', 'length', 'country'])

lakes = []
for lake in document.iterfind('lake'):
    for country in lake.get('country').split():
        area = lake.find('area')
        if area is None:
            area = np.nan
        else:
            area = float(area.text)
        lakes.append([lake.find('name').text, area, code_to_country_dict[country]])
lake_df = pd.DataFrame(lakes, columns=['name', 'area', 'country'])

airports = []
for airport in document.iterfind('airport'):
    for country in airport.get('country').split():
        elevation = airport.find('elevation')
        if elevation is None or elevation.text is None:
            continue
        else:
            elevation = float(elevation.text)
        airports.append([airport.find('name').text, elevation, code_to_country_dict[country]])
airport_df = pd.DataFrame(airports, columns=['name', 'elevation', 'country'])

max_river = river_df.loc[river_df['length'].idxmax()]
print ("Longest River:\n\t Name: {} Country: {} Length: {}".format(max_river['name'], max_river['country'], max_river['length']))

max_lake = lake_df.loc[lake_df['area'].idxmax()]
print ("Largest Lake:\n\t Name: {} Country: {} Area: {}".format(max_lake['name'], max_lake['country'], max_lake['area']))

max_airport = airport_df.loc[airport_df['elevation'].idxmax()]
print ("Highest airport:\n\t Name: {} Country: {} Elevation: {}".format(max_airport['name'], max_airport['country'], max_airport['elevation']))

Longest River:
	 Name: Amazonas Country: Colombia Length: 6448.0
Largest Lake:
	 Name: Caspian Sea Country: Russia Area: 386400.0
Highest airport:
	 Name: El Alto Intl Country: Bolivia Elevation: 4063.0
